In [1]:
import boxes
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def stats(x,y):
    
    # assuming numpy arrays
    
    params=np.polyfit(x,y,deg=1,full=True)
    n=x.shape[0]
    sx=np.sum((x-np.mean(x))**2)
    
    sse=np.sqrt(params[1][0]/sx/(n-2))
    
    return (params[0],sse)

In [11]:
def linfit(file, alg, mask_left=0, mask_right=0, write_to_file=False):
    
    bench_mark_data = boxes.io_.canonized_lb_bench(file, alg, [alg],[])
    
    lb, nb = [], []

    for s in bench_mark_data:
        size, nbs, exec_times = s

        lb.append(size)
        nb.append(np.mean(nbs))
    
    
    offset=0
    if alg=='merge':
        offset=1 # starts with lb=0 ~ gives error
        
    log_lb=np.log(np.array(lb)[offset:])
    log_nb=np.log(np.array(nb)[offset:])

    ml=mask_left # to avoid later confusion with indices
    mr=log_lb.shape[0]-mask_right
    
    while(1):
        
        if mr==log_lb.shape[0]:
            
            try: 
                # with np.polyfit line to log(lb)-log(nb) data
                
                params,sse=stats(log_lb[ml:],
                                 log_nb[ml:])
            except:
                if write_to_file:

                    with open('alg_t_db.txt','a') as f:
                        f.write('\t'+'nan\tnan'+'\n')
                        
                print('linefit caused problems '+alg+'exited without fit')
                
                return -1
        else:

            try:
                params, sse = stats(log_lb[ml:mr],
                                    log_nb[ml:mr])

            except:
                
                if write_to_file:
                    with open('alg_t_db.txt','a') as f:
                        f.write('\t'+'nan\tnan'+'\n')
                        
                print('linefit caused problems '+alg+'exited without fit')
                
                return -1
            
        fig = plt.figure()
        plt.scatter(log_lb, log_nb, label='data')
        plt.plot(log_lb[ml:mr], params[0]*log_lb[ml:mr]+params[1], c='tab:orange', label=f"fitted line (dB={round(-params[0], 3)})")
#         plt.plot([log_lb[ml],log_lb[mr]], [params[0]*log_lb[ml]+params[1],params[0]*log_lb[mr]+params[1]])
        plt.title(f"{alg} algorithm on {file[:3]}")
        plt.legend()
        if write_to_file:
            plt.savefig('linfit'+alg+'.png')
        plt.close()
        return fig, params[0]

In [5]:
import gradio as gr

In [12]:
iface = gr.Interface(fn=linfit, inputs=[gr.inputs.Textbox(placeholder="phd_cbb_benchmark.txt"), gr.inputs.Textbox(placeholder="cbb"), "number", "number", "checkbox"], 
                     outputs=[gr.outputs.Image(plot=True), "number"])
iface.launch()

Running locally at: http://127.0.0.1:7864/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


Tip: Add interpretation to your model by simply adding `interpretation="default"` to `Interface()`


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7864/', None)